###Introduction
The purpose of the analysis is to answer two questions:
- Which topics are the most/least explored per company-industry?
- Are there any groups of keywords that are explored together?

## Dataset
I will be using the processed version of the original dataset, that is dataset plus trendscores column.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv("./../output.csv")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1641482 entries, 0 to 1641481
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   company_id      1641482 non-null  int64  
 1   topic_id        1641482 non-null  int64  
 2   category_id     1641482 non-null  float64
 3   industry_id     1641482 non-null  float64
 4   week            1641482 non-null  int64  
 5   interest_level  1641482 non-null  float64
 6   trend_score     1641482 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 87.7 MB


#Exploration of Topics per Company-Industry

Here we need to keep in mind that each company only has 1 industry, however each industry might have multiple companies in it. Since we have 8717 different companies I don't think there is much point into delving into each company to see what they are interested in and what they are not (especially since we don't have interest levels for every topic). Therefore I will perform interest-level and trend analysis per industry. For company analysis I will only look at whether some company has abnormaly large interest for a topic.

In [8]:
industry_topic_agg = df.groupby(['industry_id', 'topic_id']).agg(
    avg_interest_level=pd.NamedAgg(column='interest_level', aggfunc='mean'),
    avg_trend_score=pd.NamedAgg(column='trend_score', aggfunc='mean')
).reset_index()

# I will use this aggregate table for future analysis

In [9]:
# Calculate the mean interest level for each topic across all industries and z-score for each industry's interest level
topic_mean_interest = industry_topic_agg.groupby('topic_id')['avg_interest_level'].mean().reset_index(name='mean_interest_level')

industry_topic_interest_with_mean = pd.merge(industry_topic_agg, topic_mean_interest, on='topic_id')
industry_topic_interest_with_mean['z_score'] = industry_topic_interest_with_mean.groupby('topic_id')['avg_interest_level'].transform(
    lambda x: zscore(x, ddof=1))

# Identify industries with abnormally high/low interest in topics
# Using ±2 as the threshold for significant deviation
abnormal_interest_industries = industry_topic_interest_with_mean[
    (industry_topic_interest_with_mean['z_score'] > 2) | (industry_topic_interest_with_mean['z_score'] < -2)
]
# sort by abs z-score
abnormal_interest_industries_sorted = abnormal_interest_industries.copy()
abnormal_interest_industries_sorted['abs_z_score'] = abnormal_interest_industries_sorted['z_score'].abs()

abnormal_interest_industries_sorted = abnormal_interest_industries_sorted.sort_values(by='abs_z_score', ascending=False)

most_positive_z_scores = abnormal_interest_industries_sorted.sort_values(by='z_score', ascending=False).head()
most_negative_z_scores = abnormal_interest_industries_sorted.sort_values(by='z_score', ascending=True).head()

most_positive_z_scores, most_negative_z_scores


(      industry_id  topic_id  avg_interest_level  avg_trend_score  \
 4255         58.0        70           60.000000        67.932238   
 1409        114.0        22           77.000000        60.016006   
 6203         13.0        92           50.400000        49.149840   
 6156         86.0        91           50.111111        52.392192   
 365          86.0         8           61.111111        58.875441   
 
       mean_interest_level    z_score  abs_z_score  
 4255            22.474329  10.306283    10.306283  
 1409            23.799746   9.283660     9.283660  
 6203            22.729542   8.859764     8.859764  
 6156            23.033924   8.570921     8.570921  
 365             23.777435   8.567477     8.567477  ,
       industry_id  topic_id  avg_interest_level  avg_trend_score  \
 2147         40.0        40                12.0        51.912772   
 7018        121.0       107                13.0        51.986556   
 4937         40.0        79                12.0        52

Industries with the Highest Positive Z-scores:
* Industry 58 for Topic 70: An average interest level of 60.00 with a z-score of +10.31, indicating significantly higher interest.
* Industry 114 for Topic 22: An average interest level of 77.00 with a z-score of +9.28.
* Industry 13 for Topic 92: An average interest level of 50.40 with a z-score of +8.86.
* Industry 86 for both Topic 91 and Topic 8: Showing high interest levels of 50.11 and 61.11, respectively, with z-scores of +8.57.

Industries with the Highest Negative Z-scores:
* Industry 40 for Topic 40: An average interest level of 12.00 with a z-score of -7.68, indicating significantly lower interest than average.
* Industry 121 for Topic 107: An average interest level of 13.00 with a z-score of -7.06.
* Industry 40 for Topic 79: Another instance where Industry 40 shows a low interest (12.00) with a z-score of -6.13.
* Industry 121 for Topic 39: An average interest level of 13.5 with a z-score of -6.03.
Industry 29 for Topic 90: An average interest level of 12.00 with a z-score of -6.02.

trend:

In [10]:
# Calculate the mean trend score for each topic across all industries
topic_mean_trend = industry_topic_agg.groupby('topic_id')['avg_trend_score'].mean().reset_index(name='mean_trend_score')

# Merge this mean trend score back with the aggregated data
industry_topic_trend_with_mean = pd.merge(industry_topic_agg, topic_mean_trend, on='topic_id')

# Calculate z-scores for the average trend score of each topic within each industry
industry_topic_trend_with_mean['z_score_trend'] = industry_topic_trend_with_mean.groupby('topic_id')['avg_trend_score'].transform(
    lambda x: zscore(x, ddof=1))

# Add a column for the absolute value of the z-score for sorting purposes
industry_topic_trend_with_mean['abs_z_score_trend'] = industry_topic_trend_with_mean['z_score_trend'].abs()

# Sort to find the most positive and most negative z-scores for trend scores
most_positive_z_scores_trend = industry_topic_trend_with_mean.sort_values(by='z_score_trend', ascending=False).head()
most_negative_z_scores_trend = industry_topic_trend_with_mean.sort_values(by='z_score_trend', ascending=True).head()

most_positive_z_scores_trend, most_negative_z_scores_trend

(      industry_id  topic_id  avg_interest_level  avg_trend_score  \
 4255         58.0        70           60.000000        67.932238   
 6587         13.0       100           32.833333        57.949654   
 1409        114.0        22           77.000000        60.016006   
 4024         53.0        68           44.000000        57.212670   
 2001         17.0        39           20.500000        57.577716   
 
       mean_trend_score  z_score_trend  abs_z_score_trend  
 4255         51.833089       9.894425           9.894425  
 6587         51.061576       8.133681           8.133681  
 1409         51.738036       8.092307           8.092307  
 4024         51.771296       7.907398           7.907398  
 2001         51.521376       7.808587           7.808587  ,
       industry_id  topic_id  avg_interest_level  avg_trend_score  \
 2537          9.0        47           32.666667        45.396408   
 2242         13.0        41           29.888889        43.007371   
 5649        120

Industries with the Highest Positive Z-scores for Trend Scores:
* Industry 58 for Topic 70: An average trend score of 67.93 with a z-score of +9.89, indicating a rapidly increasing interest.
* Industry 13 for Topic 100: An average trend score of 57.95 with a z-score of +8.13.
* Industry 114 for Topic 22: An average trend score of 60.02 with a z-score of +8.09.
* Industry 53 for Topic 68: An average trend score of 57.21 with a z-score of +7.91.
* Industry 17 for Topic 39: An average trend score of 57.58 with a z-score of +7.81.

Industries with the Highest Negative Z-scores for Trend Scores:
* Industry 9 for Topic 47: An average trend score of 45.40 with a z-score of -8.18, indicating a rapidly decreasing interest.
* Industry 13 for Topic 41: An average trend score of 43.01 with a z-score of -6.11.
* Industry 120 for Topic 86: An average trend score of 46.61 with a z-score of -6.01.
* Industry 86 for Topic 111: An average trend score of 44.10 with a z-score of -5.95.
* Industry 0 for Topic 69: An average trend score of 46.30 with a z-score of -5.89.

There is a lot of overlap between industry interest level and trend score analysis for high interest topics! Specifically industry 58 seems to be very interested in topic 70 and industry 114 in topic 100 (Both discrete interest levels and continuous trend scores capture this).

## Now lets explore abnormaly interested companies in a topic (compared to their industry).

In [11]:
# Calculate the mean interest level for each topic within each industry
industry_topic_mean_interest =df.groupby(['industry_id', 'topic_id'])['interest_level'].mean().reset_index(name='industry_topic_mean_interest')

# Merge this mean interest level back with the original data
data_with_industry_mean = pd.merge(df, industry_topic_mean_interest, on=['industry_id', 'topic_id'])

# Calculate z-scores for the interest level of each company-topic combination within each industry
data_with_industry_mean['z_score_company_vs_industry'] = data_with_industry_mean.groupby(['industry_id', 'topic_id'])['interest_level'].transform(
    lambda x: (x - x.mean()) / x.std(ddof=0))

# Filter to find companies with z-scores indicating significantly higher interest than their industry average
# Using a threshold of z > 2 for significant deviation
companies_with_high_interest = data_with_industry_mean[data_with_industry_mean['z_score_company_vs_industry'] > 2]

# Sort the results to highlight the most significant deviations
companies_with_high_interest_sorted = companies_with_high_interest.sort_values(by='z_score_company_vs_industry', ascending=False)

companies_with_high_interest_sorted.head()


,company_id,topic_id,category_id,industry_id,week,interest_level,trend_score,industry_topic_mean_interest,z_score_company_vs_industry
929667,1153,79,11.0,127.0,8,92.0,63.460850,21.698914,15.133868
135328,218,81,12.0,79.0,4,92.0,66.350348,21.944690,15.067009
1051539,4851,110,7.0,70.0,1,88.0,94.897959,21.805866,15.003988
982723,7339,79,11.0,10.0,7,93.0,63.830011,21.575651,14.862543
89947,5958,64,0.0,109.0,9,94.0,61.170405,21.619343,14.719215


* Company 1153 in Industry 127 for Topic 79 has a z-score of +15.13, indicating an abnormally high interest compared to the industry average.
* Company 218 in Industry 79 for Topic 81 has a z-score of +15.07.
* Company 4851 in Industry 70 for Topic 110 has a z-score of +15.00.
* Company 7339 in Industry 10 for Topic 79 has a z-score of +14.86.
* Company 5958 in Industry 109 for Topic 64 has an interest level of 94, with a z-score of +14.72.

In [12]:
# Trend scores analysis

industry_topic_mean_trend = df.groupby(['industry_id', 'topic_id'])['trend_score'].mean().reset_index(name='industry_topic_mean_trend')
data_with_industry_mean_trend = pd.merge(df, industry_topic_mean_trend, on=['industry_id', 'topic_id'])

# Calculate z-scores for the trend score of each company-topic combination within each industry
data_with_industry_mean_trend['z_score_company_vs_industry_trend'] = data_with_industry_mean_trend.groupby(['industry_id', 'topic_id'])['trend_score'].transform(
    lambda x: (x - x.mean()) / x.std(ddof=0))

# Filter to find companies with z-scores indicating significantly higher or lower trends than their industry average
# Using a threshold of z > 2
companies_with_significant_trend_deviations = data_with_industry_mean_trend[
    (data_with_industry_mean_trend['z_score_company_vs_industry_trend'] > 2)]

# Sort the results to highlight the most significant positive and negative deviations
companies_with_significant_trend_deviations_sorted = companies_with_significant_trend_deviations.sort_values(by='z_score_company_vs_industry_trend', key=abs, ascending=False)

companies_with_significant_trend_deviations_sorted.head()


,company_id,topic_id,category_id,industry_id,week,interest_level,trend_score,industry_topic_mean_trend,z_score_company_vs_industry_trend
801242,5751,63,6.0,42.0,1,89.0,95.408163,51.328048,20.750091
1065965,1256,63,6.0,70.0,1,81.0,91.326531,51.277264,20.622735
1051539,4851,110,7.0,70.0,1,88.0,94.897959,51.100543,19.395669
162855,683,63,6.0,127.0,1,82.0,91.836735,51.411710,19.152398
673680,344,64,0.0,10.0,1,82.0,91.836735,51.310140,19.007717


* Company 5751 in Industry 42 for Topic 63 has a z-score of +20.75, indicating a significantly higher trend score compared to the industry average.
* Company 1256 in Industry 70 for Topic 63 shows a z-score of +20.62.
* Company 4851 in Industry 70 for Topic 110 has a z-score of +19.40.
* Company 683 in Industry 127 for Topic 63 shows a z-score of +19.15.
* Company 344 in Industry 10 for Topic 64 has a z-score of +19.01.

There seems to be no overlap between interest_level and trend analysis for singular companies. This should be of no surprise given the volatility of interest levels and the fact that we don't have much data per company (max of 13 weeks). These different results can be used for different insights in the data.

# Exploration of keywords that are explored together

## Correlation analysis

I will start with correlation analysis with both interest levels and trends:

In [13]:
company_topic_interest = df.groupby(['company_id', 'topic_id'])['interest_level'].mean().reset_index()


pivot_table = company_topic_interest.pivot(index='company_id', columns='topic_id', values='interest_level').fillna(0)
topic_correlations = pivot_table.corr()
high_corr_pairs_interest = topic_correlations.where(np.triu(np.ones(topic_correlations.shape), k=1).astype(bool))
strong_pairs_interest = high_corr_pairs_interest.stack()
strong_pairs_interest = strong_pairs_interest[strong_pairs_interest > 0.5].sort_values(ascending=False)
strong_pairs_interest.head(10)

topic_id  topic_id
7         55          0.719978
90        92          0.673888
18        42          0.672566
55        66          0.663364
103       112         0.652181
10        54          0.646760
108       109         0.630074
68        71          0.609431
89        94          0.609385
85        90          0.589100
dtype: float64

In [14]:
company_topic_interest = df.groupby(['company_id', 'topic_id'])['trend_score'].mean().reset_index()


pivot_table = company_topic_interest.pivot(index='company_id', columns='topic_id', values='trend_score').fillna(0)
topic_correlations = pivot_table.corr()
high_corr_pairs_interest = topic_correlations.where(np.triu(np.ones(topic_correlations.shape), k=1).astype(bool))
strong_pairs_interest = high_corr_pairs_interest.stack()
strong_pairs_interest = strong_pairs_interest[strong_pairs_interest > 0.5].sort_values(ascending=False)
strong_pairs_interest.head(10)

topic_id  topic_id
7         55          0.749065
90        92          0.684474
18        42          0.682207
55        66          0.665111
68        71          0.634188
103       112         0.632650
10        54          0.625104
85        90          0.601887
108       109         0.599243
68        70          0.594609
dtype: float64

There seems to be strong correlation in both discrete interest leves and continuous trends between topics 7/55, 90/92, 18/42 and 55/66. With pair 7/55 having by far the strongest correlation in both.

## MLXtend - Mining Association Rules

In [15]:
company_week_topics = df.groupby(['company_id', 'week'])['topic_id'].apply(list).reset_index(name='topics')
transaction_lists = company_week_topics['topics'].tolist()

te = TransactionEncoder()
te_ary = te.fit(transaction_lists).transform(transaction_lists,sparse=True)
sparse_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

In [16]:
frequent_itemsets = apriori(sparse_df, min_support=0.1, use_colnames=True,low_memory=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1 )


print(rules.sort_values(by='confidence', ascending=False).head(10))

     antecedents consequents  antecedent support  consequent support  \
1948    (92, 85)        (90)            0.146823            0.333942   
963     (18, 68)        (42)            0.167512            0.335489   
1941    (81, 92)        (90)            0.140141            0.333942   
1336    (92, 30)        (90)            0.149973            0.333942   
1840    (92, 53)        (90)            0.136162            0.333942   
1906    (68, 92)        (90)            0.133310            0.333942   
1924    (75, 92)        (90)            0.133226            0.333942   
1443    (32, 92)        (90)            0.153048            0.333942   
1966   (92, 110)        (90)            0.154614            0.333942   
1875    (64, 92)        (90)            0.165424            0.333942   

       support  confidence      lift  leverage  conviction  zhangs_metric  
1948  0.117272    0.798731  2.391823  0.068242    3.309284       0.682049  
963   0.129853    0.775188  2.310619  0.073655    2.955

Here we can see 92/90 and 18/42 pairs from the correlation analysis. 85/90 is also present in corr analysis and this pair has the most confidence in rule mining (92,85)/90

# Conclusion

## High/Low interest per topic

Trend score and interest_level analysis seems to give different results for company-level interest and industry-level low interest. However, there is overlap in industry-level high interest topic analysis.

## Topic explored together

Applying correlation and rule mining techniques resulted in identifying a couple of topics that are researched together: (92,85)/90 and 18/42